# stab project 
# detection of lane lines
# ver. 0.0.1
(c) 2019 Dr.Evstigneev N.M.
 
Usage: select propper window of interest by double clicking on region points in sequence:

   (1)--(2)



(0)---------(3)

To start press 'w' or 's' depending on projection type

To change threshold press '5' or '6'

to quit press 'q'

Other parameters: see code

In addition one can add at least combinations of filters and selction of best from obvious parameters of lanes, i.e. curvature and distance between points

In [2]:
import numpy as np
import cv2

import sys
print sys.version

cv2.__version__
#WARINING! Tested for Python 2.7.14, cv2 ver. '3.4.0'
from scipy import signal
#for debug
import ipdb

2.7.14 |Anaconda custom (64-bit)| (default, Oct 16 2017, 17:29:19) 
[GCC 7.2.0]


In [3]:
pnts=[[0,0],[0,0],[0,0],[0,0]];
click=0;
set_poly = True

In [4]:
# class for 2-nd order polyninomial with given value of derivative at given point
from scipy.optimize import curve_fit

class poly_fixed_der:
    def __init__(self, zero_x_point = 0, derivative_value = 0):
        self.zero_x_point = zero_x_point
        self.derivative_value = derivative_value
        self.popt=[]
        self.poly_is_fitted = False
        
    def set_zero_x_point(self, zero_x_point_local):
        self.zero_x_point = zero_x_point_local
    
    def polynomial(self, x, a, b, c):
        return a*np.float64(x)*np.float64(x)+(self.derivative_value - 2*a*self.zero_x_point)*np.float64(x)+c
        
    def polynomial_der(self, x, a, b, c):
        return 2*a*np.float64(x)+(self.derivative_value - 2*a*self.zero_x_point)

    
    def fit(self, x_data, y_data):
        self.popt, pcov = curve_fit(self.polynomial, x_data, y_data)
        self.popt[1]=self.derivative_value - 2*self.popt[0]*self.zero_x_point
        self.poly_is_fitted = True
        
    def ret(self, x_data):
        if self.poly_is_fitted == True:
            return self.polynomial(x_data, *self.popt)
        else: 
            return 0
    
    def ret_der(self, x_data):
        if self.poly_is_fitted == True:
            return self.polynomial_der(x_data, *self.popt)
        else:
            return 0
        
    def is_fitted(self):
        return self.poly_is_fitted

In [5]:
def draw_rectangle(event,x,y,flags,param):
    global set_poly, pnts, click
    if event == cv2.EVENT_LBUTTONDBLCLK:
        if(set_poly==True):
            pnts[click]=[x,y];
            click+=1
            if(click>3):
                set_poly=False;

In [6]:
def translate_coordinates(rect_points, image_shape):
    offset = min(image_shape[0]*0.85,280)
    print rect_points[0][0] + offset
    print rect_points[3][0] + offset
    src = np.float32(rect_points)
    
#     img_change=[
#         rect_points[0],
#         [rect_points[0][0],rect_points[1][1]],
#         [rect_points[3][0],rect_points[2][1]],
#         rect_points[3]]
    
#     img_change=[
#         [0,1600],
#         [0,0],
#         [700,0],
#         [700,1600]]
    
    img_change=[
    (rect_points[0][0] + offset, image_shape[0]),
    (rect_points[0][0] + offset, 0),
    (rect_points[3][0] - offset, 0),
    (rect_points[3][0] - offset, image_shape[0])]
    dst = np.float32(img_change)
    print rect_points
    print img_change
    M = cv2.getPerspectiveTransform(src, dst) # The transformation matrix
    Minv = cv2.getPerspectiveTransform(dst, src) # Inverse transformation
    return (M,Minv)

In [7]:
# initial projection matrix
M=np.eye(3,3)
Minv=M

In [8]:
# one can use lots of different filters and their combinations
# i take only this one for demo purposes!!!
# thresholds are modified by keyboard keys

def detect_edges(img, pnts, thresh1=30, thresh2=150, thresh = 100):
    mask = np.zeros((img.shape[0], img.shape[1]), dtype="uint8")
    cv2.fillConvexPoly(mask, pnts, 255)
    masked = cv2.bitwise_and(img, img, mask=mask)
    frame = masked
    frame = cv2.threshold(masked, thresh, 255, cv2.THRESH_BINARY)[1]
    kernel_size = 5
    edges1 = cv2.GaussianBlur(frame,(kernel_size, kernel_size),0)
    #edges = cv2.Canny(edges1,thresh1,thresh2,apertureSize = 5, L2gradient=True)
    edges = cv2.Sobel(edges1,cv2.CV_64F,1,0,ksize=3)
    # edges = cv2.Scharr(edges1,cv2.CV_64F,1,0,)
    frame = cv2.threshold(edges, thresh, 255, cv2.THRESH_BINARY)[1]
    return frame
    

In [9]:
# def get_pointsP(img):
#     lines = 0
#     lines = cv2.HoughLinesP(np.uint8(img),1,np.pi/180,25)
#     img = np.zeros(img.shape)
#     if lines is not None:
#         for x1,y1,x2,y2 in lines[0]:
#             cv2.line(img,(x1,y1),(x2,y2),(10,10,10),2)
    
#     return img, lines

In [10]:
# def get_points(img):
#     lines = 0
#     lines = cv2.HoughLines(np.uint8(img),1,np.pi/180,25)
#     img = np.zeros(img.shape)
#     if lines is not None:
#         for zzz in lines:
#             rho, theta = zzz[0]
#             a = np.cos(theta)
#             b = np.sin(theta)
#             x0 = a*rho
#             y0 = b*rho
#             x1 = int(x0 + 1000*(-b))
#             y1 = int(y0 + 1000*(a))
#             x2 = int(x0 - 1000*(-b))
#             y2 = int(y0 - 1000*(a))
#             cv2.line(img,(x1,y1),(x2,y2),(100,100,100),2)
    
#     return img, lines

In [11]:
def get_peaks(img, window, center):
    axis0 = img.shape[0]
    axis1 = img.shape[1]
    center = -1
    if(center == -1):
        center = int(axis1/2)
    
    histogramL = np.sum(img[int(2*axis0/3):,:int(center)], axis=0)
    histogramR = np.sum(img[int(2*axis0/3):,int(center):], axis=0)
    peaksL = signal.find_peaks_cwt(histogramL,np.array([window]))
    peaksR = signal.find_peaks_cwt(histogramR,np.array([window]))
    if len(peaksR) > 0:
        peaksR[0] += int(axis1/2)
    
    x2 = int(0.5*(int(axis0/3)+int(2*axis0/3)))
    peaksLi = 0;
    peaksRi = 0;
    if len(peaksL) > 0:
        peaksLi=int(np.mean(peaksL))
#         cv2.circle(img, (peaksLi, x2), 10, (60,60,60) )
    if len(peaksR) > 0:
        peaksRi=int(np.mean(peaksR))
#         cv2.circle(img, (peaksRi, x2), 10, (60,60,60) )
    cornerL = 0;
    cornerR = axis1;
    if peaksLi > 0:
        cornerL = peaksLi
    if peaksRi > 0:
        cornerR = peaksRi
    
    center=int((cornerL+cornerR)*0.5)
    #cv2.circle(img, (center, x2), 15, (60,60,60) )
    imgL = img[:,:center]
    imgR = img[:,center:]
    
    return center, imgL, imgR
    

In [12]:
def fit_line(imgL, imgR, center, img, polyL, polyR):
    xL,yL = np.nonzero(imgL)
    xR,yR = np.nonzero(imgR)
    
    zL = np.polyfit(xL,yL, 1)
    zR = np.polyfit(xR,(yR+center), 1)
    PL = np.poly1d(zL)
    PR = np.poly1d(zR)
    
    xP = range(0,imgL.shape[0],10)
    yR = PR(xP)
    for x,y in zip (xP,yR):
        cv2.circle(img, (int(y),x), 10, (60,60,60))

    yL=PL(xP)
    for x,y in zip (xP,yL):
        cv2.circle(img, (int(y),x), 10, (60,60,60) ) 
    
    return img, zip(xP, yL), zip(xP, yR)

In [13]:
def fit_polynomial(imgL, imgR, center, img, polyL, polyR):
    xL,yL = np.nonzero(imgL)
    xR,yR = np.nonzero(imgR)
    
    if len(xL)>0:
        polyL.fit(xL, yL)
    if len(xR)>0:
        polyR.fit(xR, (yR+center))
    
    xP = range(0,imgL.shape[0],10)
    yR = polyR.ret(xP)
    for x,y in zip (xP,yR):
        cv2.circle(img, (int(y),x), 10, (60,60,60))

    yL = polyL.ret(xP)
    for x,y in zip (xP,yL):
        cv2.circle(img, (int(y),x), 10, (60,60,60) )        
    return img, zip(xP, yL), zip(xP,yR)

In [14]:
def fit_polynomial_by_points(img_length, XP, YP, img, polyP):
    
    polyP.fit(YP, XP)
    yP = np.int32(range(0,img_length,10))
    xP = np.int32(polyP.ret(yP))
    for x,y in zip (xP,yP):
        cv2.circle(img, (int(x),int(y)), 10, (60,60,60))
        
    return img, zip(yP, xP)

In [15]:
def put_poly_to_opigin(img, Minv, poly_points_L, poly_points_R):
    mask = np.zeros([img.shape[0], img.shape[1]])
    for x,y in poly_points_R:
        cv2.circle(mask, (int(y),x), 5, (255),5 )
    for x,y in poly_points_L:
        cv2.circle(mask, (int(y),x), 5, (255),5 ) 
    
    mask = cv2.warpPerspective(mask, Minv, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR)
    img[mask == 255] = (255, 0, 0, 0)
    
    return img

In [16]:
def get_pixel_window(img, x_center, y_center, size):
    half_size = int(size / 2)
    window = img[y_center - half_size:y_center + half_size, x_center - half_size:x_center + half_size]
    #ipdb.set_trace()
    x, y = (window.T>50).nonzero()

    x = x + x_center - half_size
    y = y + y_center - half_size

    return x, y

def detect_points_window(img, polyP, vertical_steps, center, global_img):
    pixels_per_step = int(img.shape[0] / vertical_steps)
    all_x = []
    all_y = []
    if polyP.is_fitted():
        for i in range(vertical_steps):
            #ipdb.set_trace()
            start_window = img.shape[0] - (i * pixels_per_step)
            end_window = start_window - pixels_per_step

            local_center = int((start_window + end_window) / 2)
            x = polyP.ret(local_center)-center
            x, y = get_pixel_window(img, int(x), int(local_center), pixels_per_step)
            all_x.extend(x+center)
            all_y.extend(y)

            
                
        for x_,y_ in zip(all_x,all_y):
            cv2.circle(global_img, (x_ ,y_), 2, (4),2)
        
    return all_x, all_y
    

In [17]:
#test avaliable USB devices
CHECK_CAMS = False

if(CHECK_CAMS):
    cams_test = 5
    for i in range(0, cams_test):
        cap = cv2.VideoCapture(i)
        test, frame = cap.read()
        if test:
            print("device number: "+str(i)+" -> result: "+str(test))
        
CAM_DEVICE=1

In [18]:
IMAGE_SOURCE = 'samples/test.png'

In [19]:
#VIDEO_SOURCE = 'samples/testvideo2.mp4'
VIDEO_SOURCE = 'samples/project_video.mp4'
#VIDEO_SOURCE = 'samples/Autobahn.mp4'

In [20]:
#data_source = 'image'
# datasource = 'camera'
data_source = 'video'

In [21]:
play_video = True
delta_window = 30;
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.namedWindow('origin', cv2.WINDOW_NORMAL)
polyL = poly_fixed_der(derivative_value=0)
polyR = poly_fixed_der(derivative_value=0)


if data_source=='camera':
    cap = cv2.VideoCapture(CAM_DEVICE)
elif data_source=='video':
    cap = cv2.VideoCapture(VIDEO_SOURCE)
    cv2.resizeWindow('image', 640,380)
    cv2.resizeWindow('origin', 640,380)
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(video_length)
elif data_source=='image':
    test_image = cv2.imread(IMAGE_SOURCE)
    cv2.resizeWindow('image', 640,380)
    cv2.resizeWindow('origin', 640,380)
    
cv2.setMouseCallback('image',draw_rectangle)
set_mapping=False
thresh1=50;
thresh2=130;
thresh=150;
video_fames=0;
center = -1;
while(True):
    # Capture frame-by-frame
    if data_source=='camera':
        ret, frame = cap.read()
    elif (data_source=='video')&(play_video):
        video_fames+=1
        if(video_fames==video_length-1):
            cap = cv2.VideoCapture(VIDEO_SOURCE)
            video_fames=0
            print('video restarted')
        ret, frame = cap.read()
    elif data_source=='image':
        frame = test_image
    
    # Our operations on the frame come here
    color = cv2.cvtColor(frame, cv2.COLOR_RGB2RGBA ) # cv2.COLOR_RGB2RGBA
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY) # cv2.COLOR_RGB2RGBA
    # Display the resulting frames
    
    gray1 = cv2.warpPerspective(gray, M, (gray.shape[1], gray.shape[0]), flags=cv2.INTER_LINEAR)
   
    
    
    if set_mapping:
        pnts_map = cv2.perspectiveTransform(np.array([pnts], dtype='float32'), M)
        pnts_map=np.int32(pnts_map[0])
        edges = detect_edges(gray1, pnts_map, thresh1, thresh2, thresh)
        # cv2.polylines(edges,[pnts_map],True,(255,255,255),0)
        py_min=min(pnts_map[0][1],pnts_map[3][1])
        py_max=max(pnts_map[1][1],pnts_map[2][1])
        px_min=min(pnts_map[0][0],pnts_map[1][0])
        px_max=max(pnts_map[2][0],pnts_map[3][0])     
        snip = edges#[py_max:py_min, px_min-delta_window:px_max+delta_window]
        # snip1, lines = get_pointsP(snip)
        center, imgL, imgR = get_peaks(snip, 6, center);
        x_zero_point=imgL.shape[0]
        
        polyL.set_zero_x_point(x_zero_point)
        polyR.set_zero_x_point(x_zero_point)

        XL, YL = detect_points_window(imgL, polyL, 5, 0, snip)
        XR, YR = detect_points_window(imgR, polyR, 5, center, snip)
    
        if(len(XR)<10) | (len(XL)<10):
            snip, poly_points_L, poly_points_R = fit_polynomial(imgL, imgR, center, snip, polyL, polyR)
        else:
            snip, poly_points_L = fit_polynomial_by_points(imgL.shape[0], XL, YL, snip, polyL)
            snip, poly_points_R = fit_polynomial_by_points(imgR.shape[0], XR, YR, snip, polyR)


        

        color = put_poly_to_opigin(color, Minv, poly_points_L, poly_points_R)
        gray1=snip

    
    
    pnts=np.array(pnts);
    cv2.polylines(gray1,[pnts],True,(0,255,255))
    # show images
    cv2.imshow('origin',color)
    cv2.imshow('image',gray1)
    
    #working with keys
    key = cv2.waitKey(1)
    if key & 0xFF == ord('1'):
        thresh1+=1
    elif key & 0xFF == ord('2'):
        thresh1-=1
    elif key & 0xFF == ord('3'):
        thresh2+=1
    elif key & 0xFF == ord('4'):
        thresh2-=1  
    elif key & 0xFF == ord('5'):
        thresh+=1
    elif key & 0xFF == ord('6'):
        thresh-=1  
    elif key & 0xFF == ord('q'):
        print "quit"
        break
    elif (key & 0XFF == ord('p'))&(data_source=='video'):
        play_video = not play_video
    elif (key & 0xFF == ord('s'))&(set_poly==False):
        print "projection with corrected horizon"
        pnts[2][1]=pnts[1][1]
        pnts[3][1]=pnts[0][1]
        (M,Minv) = translate_coordinates(pnts, color.shape);
        set_mapping = True
    elif (key & 0xFF == ord('w'))&(set_poly==False):
        print "setting projection"
        (M,Minv) = translate_coordinates(pnts, color.shape);
        set_mapping = True
    elif key & 0xFF == ord('i'):
        print "setting inverse projection"
        E=Minv;
        Minv=M;
        M=E;
    elif key & 0xFF == ord('r'):
        print "reset"
        M=np.eye(3,3)
        Minv=M;
        click=0;
        set_poly=True;
        pnts = [[0,0],[0,0],[0,0],[0,0]]
        set_mapping=False
    elif key & 0xFF == ord('e'):
        print "identity projection"
        M=np.eye(3,3)
        Minv=M;
        
    
# When everything done, release the capture
if data_source!='image':
    cap.release()

cv2.destroyAllWindows()


1260
setting projection
531
1440
[[ 251  644]
 [ 548  451]
 [ 698  448]
 [1160  644]]
[(531, 720), (531, 0), (880, 0), (880, 720)]


/home/noctum/anaconda2/lib/python2.7/site-packages/scipy/optimize/minpack.py:785: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


projection with corrected horizon
531
1440
[[ 251  644]
 [ 548  451]
 [ 698  451]
 [1160  644]]
[(531, 720), (531, 0), (880, 0), (880, 720)]
quit
